<center>

# First Assignment of RT2 (Part Two: Jupyter Notebook)

</center>

### Import all the necessary libraries and initialization of the global variables

In [ ]:
# import all the necessary libraries
%matplotlib notebook
import tf
import sys
import rospy
import select
import actionlib
import numpy as np
import actionlib.msg
import jupyros as jr
import ipywidgets as widgets
import assignment_2_2022.msg 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from std_srvs.srv import *
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from assignment_2_2022.msg import Pos_vel
from matplotlib.animation import FuncAnimation
from geometry_msgs.msg import Point, Pose, Twist
from tf.transformations import quaternion_matrix
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox


# initialize the node 
rospy.init_node('input_from_user')

# initialize the publisher to send a msg (using the velocity and the position as parameters)
pub = rospy.Publisher("/Position_velocity", Pos_vel, queue_size = 1)

# initialize the global variable necessary for the cell below
actual_pos_x = 0
actual_pos_y = 0
delay_actual_pos = 0

n_goal_deleted = 0
n_goal_set = 0

min_distance_obstacle = 0
delay_min_distance_obstacle = 0

### Client initialization for sending goals

In [ ]:
# create the action client
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
# wait for the server to be started
client.wait_for_server()

### Plot of the robot's path and goals (set, reached, deleted) 

In [ ]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'mo')
        self.x_data, self.y_data = [] , []
    def plot_init(self):
        self.ax.set_xlim(10, -10)
        self.ax.set_ylim(10, -10)
        self.ax.grid(True)
        return self.ln
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    def draw_point(self, x, y, color):
        self.ax.plot(x, y, color, marker='X')
    def add_legend(self):
        legend_labels = ['Goal Deleted', 'Goal Reached', 'Next Goal']
        colors = ['red', 'limegreen', 'dodgerblue']
        legend_elements = [
                            plt.Line2D([0], [0], marker='X', color=color, label=label, linestyle='None')
                            for color, label in zip(colors, legend_labels)
                          ]
        self.ax.legend(handles=legend_elements)
        
print("PLOT OF THE ROBOT'S PATH AND GOALS (SET; REACHED, DELETED): ")
vis = Visualiser()
    
rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)

def goal_status_function(msg):
    global positionX,positionY,vis
    # get the status of the result from the message
    status = msg.status.status

    # if status is equal 2, the goal was deleted
    if status == 2:
        vis.draw_point(positionX.value, positionY.value, 'red')
    # if status is equal 3, the goal was reached
    elif status == 3:
        vis.draw_point(positionX.value, positionY.value, 'limegreen')

    
sub_result = rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult, goal_status_function)
vis.add_legend()
plt.show(block=True)

### Actual position of the robot (updated in real time)

In [ ]:
def publish_values(msg):
    global delay_actual_pos
    delay_actual_pos += 1
    # get the postion from the message
    pos = msg.pose.pose.position

    # get the twist from the message
    velocity = msg.twist.twist.linear

    # create custom message (of the type corresponding to the file in the msg folder)
    position_and_velocity = Pos_vel()

    # assign the parameters of the custom message
    position_and_velocity.x = pos.x
    position_and_velocity.y = pos.y
    position_and_velocity.velX = velocity.x
    position_and_velocity.velY = velocity.y
    
    global actual_pos_x
    global actual_pos_y
    actual_pos_x = pos.x
    actual_pos_y = pos.y
    if delay_actual_pos >= 15:
        text_box_actual_pos_x.value = f'{actual_pos_x:.6f}'
        text_box_actual_pos_y.value = f'{actual_pos_y:.6f}'
        delay_actual_pos = 0

    # Publish the custom message
    pub.publish(position_and_velocity)

#subscribe to the topic /odom
text_box_actual_pos_x = widgets.FloatText(description="Act pos x", value=actual_pos_x, disabled=True, continuous_update=True)
text_box_actual_pos_y = widgets.FloatText(description="Act pos y", value=actual_pos_y, disabled=True, continuous_update=True)
rospy.Subscriber("/odom", Odometry, publish_values)
print("ACTUAL POSITION OF THE ROBOT, UPDATED IN REAL TIME: ")
display(text_box_actual_pos_x, text_box_actual_pos_y)

### Possibility to send or delete a goal using coordinates

In [ ]:
print("ENTER THE COORDINATES (x,y) BELOW THAT THE ROBOT MUST REACH")
positionX = widgets.FloatText(description = "x", layout=Layout(width='25%', align="center", grid_area='pX'))
positionY = widgets.FloatText(description = "y", layout=Layout(width='25%', align="center", grid_area='pY'))
send_btn = widgets.Button(description="Send Goal!", button_style='primary', layout=Layout(width='25%', align="center",
grid_area='b1'), icon='check')
canc_btn = widgets.Button(description="Cancel Goal!", button_style='danger', layout=Layout(width='25%', align="center",
grid_area='b2'))
box = VBox([HBox([positionX, positionY]), HBox([send_btn, canc_btn])])
display(box)

def send_goal_function(button):
    global n_goal_set
    global text_box_number_set_goal
    # create the goal to send to the server
    goal = assignment_2_2022.msg.PlanningGoal()
    goal.target_pose.pose.position.x = positionX.value
    goal.target_pose.pose.position.y = positionY.value
    vis.draw_point(positionX.value, positionY.value, 'dodgerblue')
    # send the goal to the action server
    client.send_goal(goal)
    n_goal_set += 1
    text_box_number_set_goal.value = n_goal_set

def deleted_goal_function(button):
    global n_goal_deleted
    global n_goal_set
    global text_box_number_del_goal
    if n_goal_set > n_goal_deleted:
        client.cancel_goal()
        n_goal_deleted += 1
        text_box_number_del_goal.value = n_goal_deleted
    
def draw_pie_plot(valore1, valore2):
    labels = ['Goal Set', 'Goal Deleted']
    mycolors = ["dodgerblue", "red"]
    sizes = [valore1, valore2]
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors = mycolors)
    ax.axis('equal')
    plt.title('Number of set and deleted goals')
    plt.show()

### Number of set and deleted goals with their corresponding percentage pie plot

In [ ]:
text_box_number_set_goal = widgets.IntText(description="N° set goals", value=n_goal_set, disabled=True)
text_box_number_del_goal = widgets.IntText(description="N° del goals", value=n_goal_deleted, disabled=True)

interactive_plot = widgets.interactive(draw_pie_plot, valore1=text_box_number_set_goal, valore2=text_box_number_del_goal)
display(interactive_plot)
    
send_btn.on_click(send_goal_function)
canc_btn.on_click(deleted_goal_function)

### Distance from the nearest obstacle

In [ ]:
text_box_rt_min_distance_obstacle = widgets.FloatText(description="", value=min_distance_obstacle, disabled=True)
text_box_act_min_distance_obstacle = widgets.FloatText(description="", value=min_distance_obstacle, disabled=True)
def min_obstacle_distance(msg):
    global min_distance_obstacle
    global delay_min_distance_obstacle 
    delay_min_distance_obstacle += 1
    min_distance_obstacle = msg.ranges[0]
    for i in msg.ranges:
        if  delay_min_distance_obstacle >= 25 and i < min_distance_obstacle:
            min_distance_obstacle = i
            text_box_rt_min_distance_obstacle.value = f'{min_distance_obstacle:.6f}'
            delay_min_distance_obstacle = 0

rospy.Subscriber("/scan", LaserScan, min_obstacle_distance)
print("DISTANCE FROM THE NEAREST OBSTACLE, UPDATED IN REAL TIME: ")
display(text_box_rt_min_distance_obstacle)
print("CLICK THE BUTTON TO KNOW THE ISTANTANEOUS DISTANCE FROM THE NEAREST OBSTACLE: ")
act_min_obstcale_distance_btn = widgets.Button(description="UPDATE MIN DISTANCE", button_style='primary', 
                                               layout=Layout(width='25%', align="center", grid_area='b3'))
box2 = HBox([act_min_obstcale_distance_btn, text_box_act_min_distance_obstacle])
display(box2)

def update_actual_min_obstacle_distance_function(button):
    text_box_act_min_distance_obstacle.value = f'{min_distance_obstacle:.6f}'

act_min_obstcale_distance_btn.on_click(update_actual_min_obstacle_distance_function)